# NPS анализ


**Задача:** определить текущий уровень потребительской лояльности среди клиентов телекоммуникационной компании из России.

## Подключение к базе

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = 'datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка данных

In [3]:
query = """
SELECT usr.user_id,
       usr.lt_day,
       CASE
           WHEN usr.lt_day <= 365 THEN TRUE
           ELSE FALSE
       END AS is_new,
       usr.age,
       CASE 
           WHEN usr.gender_segment = 1 THEN 'female'
           WHEN usr.gender_segment = 0 THEN 'male'
           ELSE 'unknown' 
        END AS gender_segment,
        usr.os_name,
        usr.cpe_type_name,
        lcl.country,
        lcl.city,
        age_sgm.title AS age_segment,
        tr_sgm.title AS traffic_segment,
        lt_sgm.title AS lifetime_segment,
        usr.nps_score,
        CASE
            WHEN nps_score >= 9 THEN 'promoters'
            WHEN nps_score >= 7 THEN 'passives'
            ELSE 'detractors'
        END AS nps_group
FROM user AS usr
LEFT OUTER JOIN location AS lcl ON usr.location_id = lcl.location_id
LEFT OUTER JOIN age_segment AS age_sgm ON usr.age_gr_id = age_sgm.age_gr_id 
LEFT OUTER JOIN traffic_segment AS tr_sgm ON usr.tr_gr_id = tr_sgm.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS lt_sgm ON usr.lt_gr_id = lt_sgm.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,0,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,0,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters


In [6]:
df.to_csv('telecomm_csi_tableau_.csv', index=False)

## Дашборд

In [7]:
# https://public.tableau.com/views/NPS_16974740581420/NPS_2?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link